---
## Bigdl-nano Resnet example on CIFAR10 dataset
---
This example illustrates how to apply bigdl-nano optimizations on a image recognition case based on pytorch-lightning framework. The basic image recognition module is implemented with Lightning and trained on [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) image recognition Benchmark dataset. 

In [1]:
import os
from time import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.functional import accuracy
from bigdl.nano.pytorch.trainer import Trainer
from bigdl.nano.pytorch.vision import transforms

### CIFAR10 Data Module
---
Import the existing data module from bolts and modify the train and test transforms.
You could access [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) for a view of the whole dataset.


In [2]:
def prepare_data(data_path, batch_size, num_workers):
    train_transforms = transforms.Compose(
        [
            transforms.RandomCrop(32, 4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            cifar10_normalization()
        ]
    )

    test_transforms = transforms.Compose(
        [
            transforms.ToTensor(),
            cifar10_normalization()
        ]
    )
    cifar10_dm = CIFAR10DataModule(
        data_dir=data_path,
        batch_size=batch_size,
        num_workers=num_workers,
        train_transforms=train_transforms,
        test_transforms=test_transforms,
        val_transforms=test_transforms
    )
    return cifar10_dm

### Resnet
---
Modify the pre-existing Resnet architecture from TorchVision. The pre-existing architecture is based on ImageNet images (224x224) as input. So we need to modify it for CIFAR10 images (32x32).

In [3]:
def create_model():
    model = torchvision.models.resnet18(pretrained=False, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    return model

### Lightning Module
---
Check out the [configure_optimizers](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html#configure-optimizers) method to use custom Learning Rate schedulers. The OneCycleLR with SGD will get you to around 92-93% accuracy in 20-30 epochs and 93-94% accuracy in 40-50 epochs. Feel free to experiment with different LR schedules from https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate

In [4]:
class LitResnet(LightningModule):
    def __init__(self, learning_rate=0.05):
        super().__init__()

        self.save_hyperparameters()
        self.model = create_model()

    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss)
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_acc", acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.hparams.learning_rate,
            momentum=0.9,
            weight_decay=5e-4,
        )
        steps_per_epoch = 45000
        scheduler_dict = {
            "scheduler": OneCycleLR(
                optimizer,
                0.1,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=steps_per_epoch,
            ),
            "interval": "step",
        }
        return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}

In [5]:
seed_everything(7)
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 64
NUM_WORKERS = int(os.cpu_count() / 2)
data_module = prepare_data(PATH_DATASETS, BATCH_SIZE, NUM_WORKERS)

Global seed set to 7


### Train
Use Trainer from bigdl.nano.pytorch.trainer for BigDL-Nano pytorch.

This Trainer extends PyTorch Lightning Trainer by adding various options to accelerate pytorch training.

```
    :param num_processes: number of processes in distributed training. default: 4.
    :param use_ipex: whether we use ipex as accelerator for trainer. default: True.
    :param cpu_for_each_process: A list of length `num_processes`, each containing a list of
            indices of cpus each process will be using. default: None, and the cpu will be
            automatically and evenly distributed among processes.
```
The next few cells show examples of different parameters.
#### Single Process
---

In [6]:
model = LitResnet(learning_rate=0.05)
model.datamodule = data_module
basic_trainer = Trainer(num_processes = 1,
                  use_ipex = False,
                  progress_bar_refresh_rate=10,
                  max_epochs=30,
                  logger=TensorBoardLogger("lightning_logs/", name="basic"),
                  callbacks=[LearningRateMonitor(logging_interval="step")])
start = time()
basic_trainer.fit(model, datamodule=data_module)
basic_fit_time = time() - start
outputs = basic_trainer.test(model, datamodule=data_module)
basic_acc = outputs[0]['test_acc'] * 100

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Files already downloaded and verified
Files already downloaded and verified



  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.696    Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 7


Training: -1it [00:00, ?it/s]

/home/mingzhi/anaconda3/envs/forOpenvino/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.2045000046491623, 'test_loss': 2.144069194793701}
--------------------------------------------------------------------------------


### multi processes
---

In [9]:
model = LitResnet(learning_rate=0.05)
model.datamodule = data_module
mulit_trainer = Trainer(num_processes=2,
                        use_ipex = False,
                        distributed_backend="subprocess",
                        progress_bar_refresh_rate=10,
                        max_epochs=3,
                        logger=TensorBoardLogger("lightning_logs/", name="mulit"),
                        callbacks=[LearningRateMonitor(logging_interval="step")])
start = time()
mulit_trainer.fit(model, datamodule=data_module)
mulit_fit_time = time() - start
outputs = mulit_trainer.test(model, datamodule=data_module)
mulit_acc = outputs[0]['test_acc'] * 100

/home/mingzhi/anaconda3/envs/forOpenvino/lib/python3.9/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py:74: LightningDeprecationWarning: Argument `num_nodes` in `DDPSpawnPlugin` is deprecated in v1.4, and will be removed in v1.6. Notice that it will be overriden by the trainer setting.
  rank_zero_deprecation(
/home/mingzhi/anaconda3/envs/forOpenvino/lib/python3.9/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py:80: LightningDeprecationWarning: Argument `sync_batchnorm` in `DDPSpawnPlugin` is deprecated in v1.4, and will be removed in v1.6. Notice that it will be overriden by the trainer setting.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


Files already downloaded and verified
Files already downloaded and verified


initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/2
initializing ddp: GLOBAL_RANK: 1, MEMBER: 2/2
----------------------------------------------------------------------------------------------------
distributed_backend=gloo
All DDP processes registered. Starting ddp with 2 processes
----------------------------------------------------------------------------------------------------


  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
11.2 M    Trainable params
0         Non-trainable params
11.2 M    Total params
44.696    Total estimated model params size (MB)


Epoch 0:   0%|          | 0/392 [00:00<00:00, 5433.04it/s]    

[W reducer.cpp:1158] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later iterations to have unused parameters. (function operator())
[W reducer.cpp:1158] Warning: find_unused_parameters=True was specified in DDP constructor, but did not find any unused parameters in the forward pass. This flag results in an extra traversal of the autograd graph every iteration,  which can adversely affect performance. If your model indeed never has any unused parameters in the forward pass, consider turning this flag off. Note that this warning may be a false positive if your model has flow control causing later 

Epoch 0:  82%|████████▏ | 320/392 [18:31<04:09,  3.46s/it, loss=1.43, v_num=11]
Validating: 0it [00:00, ?it/s]
Epoch 1:  31%|███       | 120/392 [07:11<16:10,  3.57s/it, loss=1.28, v_num=11, val_loss=1.430, val_acc=0.487]

/home/mingzhi/anaconda3/envs/forOpenvino/lib/python3.9/site-packages/pytorch_lightning/trainer/trainer.py:1047: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
/home/mingzhi/anaconda3/envs/forOpenvino/lib/python3.9/site-packages/pytorch_lightning/plugins/training_type/ddp_spawn.py:288: UserWarning: cleaning up ddp environment...
  rank_zero_warn("cleaning up ddp environment...")


KeyboardInterrupt: 

### Train with IPEX
---

In [ ]:
model = LitResnet(learning_rate=0.05)
model.datamodule = data_module
ipex_trainer = Trainer(num_processes=2,
                       use_ipex=True,
                       progress_bar_refresh_rate=10,
                       max_epochs=30,
                       logger=TensorBoardLogger("lightning_logs/", name="ipex"),
                       callbacks=[LearningRateMonitor(logging_interval="step")])
start = time()
ipex_trainer.fit(model, datamodule=data_module)
ipex_fit_time = time() - start
outputs = ipex_trainer.test(model, datamodule=data_module)
ipex_acc = outputs[0]['test_acc'] * 100

In [ ]:
template = """
|    Precision   | Fit Time(s)       | Accuracy(%) |
|      Basic     |       {:5.2f}       |    {:5.2f}    |
|      Mulit     |       {:5.2f}       |    {:5.2f}    |
|      Ipex      |       {:5.2f}       |    {:5.2f}    |
"""
summary = template.format(
    basic_fit_time, basic_acc,
    mulit_fit_time, mulit_acc,
    ipex_fit_time, ipex_acc
)
print(summary)